In [1]:
import numpy as np
import gym
import time
import math

In [13]:
env = gym.make("CartPole-v1")
action_space = env.action_space.n
print(action_space)

2


In [16]:
alpha = 0.1 #learning rate
max_iterations = 100000
gamma = 0.995

total = 0
tot_reward = 0
prev_reward = 0

observation = [30, 30, 50, 50]
array_win = np.array([0.25, 0.25, 0.01, 0.1])

epsilon = 1
epsilon_decay = 0.99995

In [17]:
q_table = np.random.uniform(low=0, high=1, size=(observation + [env.action_space.n]))
q_table.shape

(30, 30, 50, 50, 2)

In [5]:
magic_array =  np.array([15,10,1,10])
def get_discrete_state(state):
    discrete_state = state/array_win + magic_array
    return tuple(discrete_state.astype(int))

In [18]:
for i in range(max_iterations):
    t0 = time.time()
    discrete_state = get_discrete_state(env.reset())
    done = False

    reward = 0

    if (i % 2000 == 0):
        print(f"Episode: {i}")
    
    while(not(done)):
        
        if np.random.random() > epsilon:
            action = np.argmax(q_table[discrete_state])
        else:
            action = np.random.randint(0, action_space)

        new_state, step_reward, done, info = env.step(action)

        reward += step_reward

        new_discrete_state = get_discrete_state(new_state)

        if (i % 2000 == 0):
            env.render()
        
        if not done:
            max_future_q = np.max(q_table[new_discrete_state])
            current_q = q_table[discrete_state + (action,)]
            
            new_q = (1 - alpha) * current_q + alpha * (reward + gamma * max_future_q)

            q_table[discrete_state + (action,)] = new_q
        
        discrete_state = new_discrete_state

    if epsilon > 0.05: 
        if reward > prev_reward and i > 10000:
            epsilon = math.pow(epsilon_decay, i - 10000)

            if i % 500 == 0:
                print("Epsilon: " + str(epsilon))
    
    t1 = time.time()
    iteration_time = t1-t0
    total += iteration_time

    tot_reward += reward
    prev_reward = reward

    if i % 1000 == 0: 
        mean = total / 1000
        print("Time Average: " + str(mean))
        total = 0

        mean_reward = tot_reward / 1000
        print("Mean Reward: " + str(mean_reward))
        tot_reward = 0



Episode: 0
Time Average: 0.0003339965343475342
Mean Reward: 0.016
Time Average: 0.002742120981216431
Mean Reward: 22.943
Episode: 2000
Time Average: 0.0032657458782196047
Mean Reward: 21.939
Time Average: 0.0015198793411254883
Mean Reward: 21.815
Episode: 4000
Time Average: 0.002586453676223755
Mean Reward: 22.592
Time Average: 0.0016990513801574706
Mean Reward: 22.174
Episode: 6000
Time Average: 0.002977961540222168
Mean Reward: 21.749
Time Average: 0.0015129806995391845
Mean Reward: 22.302
Episode: 8000
Time Average: 0.0017751960754394532
Mean Reward: 21.616
Time Average: 0.0022729485034942627
Mean Reward: 22.378
Episode: 10000
Time Average: 0.0026828856468200683
Mean Reward: 22.524
Time Average: 0.0018529932498931884
Mean Reward: 22.637
Episode: 12000
Time Average: 0.002387116193771362
Mean Reward: 23.572
Time Average: 0.002303032636642456
Mean Reward: 24.68
Epsilon: 0.8394533480303666
Episode: 14000
Time Average: 0.002741000175476074
Mean Reward: 26.727
Epsilon: 0.7985117269685725


In [14]:
env.close()